# Calcul des "*Golden Scores*"

## I - Introduction

L'idée principale du projet est d'évaluer des métriques qui mesurent la performance de systèmes de traduction. Un des premiers critères à considérer est leur corrélation avec le jugement humain. Les données du WMT22 nous permettent d'affecter un score humain à chaque traduction candidate des systèmes considérés via le calcul des scores **MQM** (Multidimensional Quality Metrics). Le **MQM** est un type de **Golden Score** car il correspond à un score basé sur les erreurs identifiées par les experts humains.

Nous disposons de trois paires de langages source (**sl** pour "source langage") et langages cibles (**tl** pour "target langage") :

- *English-German* : **ende**
- *Chinese-English* : **zhen**
- *English-Russian* : **enru**

Pour chacune de ces trois paires, nous disposons de deux bases de données issues du GitHub Google MQM Human Evaluation (https://github.com/google/wmt-mqm-human-evaluation) :

1 - **Base de données "Liste Candidats"** : répertorie les traductions de différents modèles de langage (**hyp** pour hypothèse), les phrases source (**source**), les traductions de référence établies par des experts (**ref** pour référence). 

2 - **Base de données "Correction des Candidats"** : répertorie l'ensemble des erreurs faites par les modèles de traduction. Un expert humain a annoté pour chaque traduction candidate de chaque système les erreurs de traduction, en indiquant pour chaque erreur son type et sa sévérité, c'est à dire s'il s'agit d'une faute importante ou non. 

Ici, le MQM pour chaque traduction candidate se calcule comme la somme sur toutes les erreurs identifiées des poids associés à leur sévérité. Les poids sont donc à percevoir comme des pénalités, où le score de la traduction se voit décrémenté d'une certaine valeur pour chaque erreur commise. Formellement, on peut écrire : 

\begin{equation}
MQM_{hyp} = - \sum_{error \in Errors} w_{error} 
\end{equation}

Le barème des poids par type d'erreur provient du concours WMT 2021, et deux tableaux récapitulatifs provenant de l'article *"Results of the WMT21 Metrics Shared Task: Evaluating Metrics with Expert-based Human Evaluations on TED and News Domain"* (https://aclanthology.org/2021.wmt-1.73/) dresse une liste exhaustive de ceux ci. La table 2 (Google MQM) est utilisée pour les paires *English-German* : **ende** et *Chinese-English* : **zhen**. La table 3 (Unlabel MQM) est utilisée pour la paire *English-Russian* : **enru**.

### I.1 - *English-German* (**ende**) et *Chinese-English* (**zhen**) : **Google's MQM error**

<img src="./figures/google_mqm_score.png" alt="Employee data" width="250" height="250" title="Employee Data title">

### I.2 - *English-Russian* (**enru**) : **Unlabel's MQM error**

<img src="./figures/unlabel_mqm_score.png" alt="Employee data" width="300" height="300" title="Employee Data title">

### I.3 - Classe WMT22 pour Calcul des scores

L'ensemble des fonctions permettant de calculer les MQM scores pour les trois paires de traductions sont à retrouver dans **/code/data_processing** : 
- **MQM_google_score** : calcul des MQM scores pour la partie I.1
- **MQM_unlabel_score** : calcul des MQM scores pour la partie I.2
- **data_golden_scores** : formation des bases de données de traductions annotées des scores MQM, et enregistrement sous le nom WMT22_w_golden_scores_sltl
- **read_data** : lecture de la base de données déjà annotée des scores (pour éviter le recalcul des scores à chaque session de travail).

## II - Création base avec Golden Scores

In [126]:
# IMPORTS :
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import seaborn as sns
import sys

# FUNCTIONS / CLASSES :
%load_ext autoreload
%autoreload 2
sys.path.append("./code")
from data_processing import WMT22

## DICTIONNAIRE DES COUPLES DE TRADUCTIONS :
sl_tls = {
    "ende" : "English-German",
    "enru" : "English-Russian",
    "zhen" : "Chinese-English" 
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
## PARCOURS DES COUPLES DE TRADUCTION POSSIBLES :
for key in sl_tls.keys() :
    
    val = sl_tls[key]
    
    ## ACCES AUX DONNEES DE TRAIN ET DEV :
    df = WMT22.data_golden_scores(sl_tl = key)
    
    ## NOMBRE D'OBSERVATIONS :
    print(val , " : " , len(df) , " observations" )
    


English-German  :  32190  observations


/Users/henriup/Desktop/project_nlp/./code/data_processing.py:145: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_rates = pd.read_table("./data/generalMT2022/" + sl_tl + "/mqm_generalMT2022_" + sl_tl + ".tsv",


English-Russian  :  32348  observations
Chinese-English  :  35188  observations


## III - Lecture des données

In [136]:
## PARCOURS DES COUPLES DE TRADUCTION POSSIBLES :
for key in sl_tls.keys() :
    
    val = sl_tls[key]
    
    ## ACCES AUX DONNEES DE TRAIN ET DEV :
    df = WMT22.read_data(sl_tl = key)
    
    ## NOMBRE D'OBSERVATIONS :
    print(val , " : " , len(df) , " observations" )
    
    print(df.sample(10)[["ref", "hyp", "score"]])
    
    print("\n")

English-German  :  32190  observations
                                                     ref  \
9171   - Schließen Sie das Wandladegerät (nicht im Li...   
15065  Melden Sie sich einfach bei Ihrem Konto an und...   
4446        Wahrscheinlich haben wir, wonach Sie suchen!   
28069  Einige von uns müssen zumindest etwas Komfort ...   
22613                      Wir danken für Ihren Einkauf.   
15446  Das kann das Ergebnis ihrer oder meiner Komple...   
27857  Um Ihren #PRS_ORG# auf die Werkseinstellungen ...   
16159  Ein bahnbrechendes Element ihres Projekts ist ...   
25984  Das kann ihnen ernsthaft schaden und ein Tier ...   
1143   Drücken und halten Sie die Power-Taste, bis si...   

                                                     hyp  score  
9171   - Schließen Sie das Ladegerät (nicht im Liefer...    0.0  
15065  Melden Sie sich einfach bei Ihrem Konto an und...    0.0  
4446    Wir haben wahrscheinlich das, wonach Sie suchen!    0.0  
28069  Einige von uns müssen zuminde